In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/RAG
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/RAG
/content/drive/MyDrive/RAG


In [3]:
!pip uninstall -y faiss faiss-cpu faiss-gpu
!pip install faiss-cpu

  Using cached faiss_cpu-1.9.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.9.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.5 MB)


In [4]:
import faiss

In [5]:
!pip install datasets

In [6]:
import os
# from modeling_rag import RagTokenForGeneration
# from retrieval_rag import RagRetriever
# from tokenization_rag import RagTokenizer
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

dataset = load_dataset("Shannnh/small-dataset-test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
import faiss
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import pandas as pd

# 验证 FAISS 是否安装成功
try:
    import faiss
    print("FAISS version:", faiss.__version__)
except ImportError as e:
    print("FAISS is not installed correctly:", e)

dataset = load_dataset("Shannnh/small-dataset-test")

FAISS version: 1.9.0


In [11]:
import faiss
import numpy as np

# 假设 'embeddings' 列包含预计算的嵌入向量
embeddings = np.array(dataset['train']['embeddings']).astype('float32')  # shape (num_passages, vector_dim)

# 检查嵌入的维度
dimension = embeddings.shape[1]
print(f"Embedding dimension: {dimension}")

# 创建 FAISS 索引
index = faiss.IndexFlatL2(dimension)  # 使用 L2 距离度量
index.add(embeddings)  # 添加嵌入到索引中

# 保存 FAISS 索引到磁盘
faiss.write_index(index, "faiss_index")

Embedding dimension: 768


In [14]:
# 提取 'text' 和 'title' 列并创建知识库 DataFrame
passages_df = pd.DataFrame({
    "text": dataset['train']['text'],  # 确保列名正确
    "title": dataset['train']['title']  # 如果需要，可以保留 'title' 列
})

# 转换为 Hugging Face Dataset
passage_dataset = Dataset.from_pandas(passages_df)

# 验证 'embeddings' 列的格式
embeddings = np.array(dataset['train']['embeddings']).astype('float32')  # 确保类型为 float32
print(f"Embedding shape: {embeddings.shape}")

Embedding shape: (3, 768)


In [15]:
# 保存知识库数据集到磁盘
passage_dataset.save_to_disk("passage_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

In [16]:
import faiss

# 获取嵌入维度
dimension = embeddings.shape[1]
print(f"Embedding dimension: {dimension}")

# 创建 FAISS 索引
index = faiss.IndexFlatL2(dimension)  # 使用 L2 距离度量
index.add(embeddings)  # 添加嵌入到索引中

# 保存 FAISS 索引到磁盘
faiss.write_index(index, "faiss_index")

Embedding dimension: 768


In [18]:
model_name = "facebook/rag-sequence-nq"
tokenizer = RagTokenizer.from_pretrained(model_name)

retriever = RagRetriever.from_pretrained(
    model_name,
    index_name="custom",            # 使用自定义索引
    dataset_path="passage_dataset", # 知识库数据集的路径
    index_path="faiss_index",       # FAISS 索引文件的路径
    repo_type="dataset"             # 显式指定 repo_type
)

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


...

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may res

ValueError: Please provide `dataset_path` and `index_path` after calling `dataset.save_to_disk(dataset_path)` and `dataset.get_index('embeddings').save(index_path)`.

In [ ]:
model = RagTokenForGeneration.from_pretrained(model_name, retriever=retriever)

# 加载训练和验证数据集（替换为实际训练和验证数据集）
training_dataset = dataset["train"]  # 替换实际训练数据集
validation_dataset = dataset["validation"]  # 替换实际验证数据集

# 配置训练参数
training_args = TrainingArguments(
    output_dir="./rag_finetune",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=10,
    overwrite_output_dir=True
)

# 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

# 开始训练
trainer.train()